## Goal: determining attraction / retention

An important characteristic of a how a community evolves is how contributors are joining and leaving. Determining when contributors leave is not easy (as they could be on a temporary leave, but coming back later), but if after a certain period they are still inactive, it is very likely they can be considered 'gone'. With this definition, the evolution of attraction and retention, and its difference (net gain of developers, which can be negative) can be computed.

These data could be determined for each of the specific contributors groups defined in the first goal. 

This goal can be refined in the following questions:

**Questions**:

* How many contributors are joining the community?
* How many contributos are no longer active (leaving) in the community?
* How is the attraction / retention ratio, and the net gain of contributors, over time?

To answer these questions, the following metrics can be used:

**Metrics**:

* Number of contributors joining the community over time (attracted)
* Number of contirbutors leaving (becoming inactive) over time
* Number of contributors not leaving (retained) over time

These metrics can be computed for each of the "cohorts", defined as the groups of contributors joining during a certain period of time (for example, during each year). Some of these metrics will be computed for the speficied contributor groups, over time.

# Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.

This section includes common code to manage and plot data. Queries will be available at the corresponding section.

**TODO**: Add bot and merges filtering.

**TODO** : provide plots similar to:

https://analytics.mozilla.community/edit/app/kibana#/dashboard/Community-Analytics-Demographics

In [1]:
from datetime import datetime
import pandas

import plotly as plotly
import plotly.graph_objs as go

import util as ut

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

In [2]:
def create_search(source):
    s = Search(using=es_conn, index=source)
    # TODO: Add bot and merges filtering.
    #s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
    #s.params(timeout=100)
    return s

In [3]:
def get_authors_df(result, author_bucket_field):        

    # Get a dataframe with each author and their first commit
    buckets_result = result['aggregations'][author_bucket_field]['buckets']

    buckets = []
    for bucket_author in buckets_result:
        author = bucket_author['key']

        first = bucket_author['first']['hits']['hits'][0]
        first_commit = first['sort'][0]/1000
        last_commit = bucket_author['last_commit']['value']/1000
        org_name = first['_source']['author_org_name']
        uuid = first['_source']['author_uuid']
        buckets.append({
                'first_commit': datetime.utcfromtimestamp(first_commit),
                'last_commit': datetime.utcfromtimestamp(last_commit),
                'author': author,
                'uuid': uuid,
                'org': org_name
        })
    authors_df = pandas.DataFrame.from_records(buckets)
    authors_df.sort_values(by='first_commit', ascending=False,
                            inplace=True)
    return authors_df

def get_active_authors_df(result, author_bucket_field, year):
    """Returns a dataframe with first and last commit of those authors
    whose last commit was made within a given year"""

    # Get a dataframe with each author and their first commit
    buckets_result = result['aggregations'][author_bucket_field]['buckets']

    buckets = []
    for bucket_author in buckets_result:
        author = bucket_author['key']

        first = bucket_author['first']['hits']['hits'][0]
        first_commit = first['sort'][0]/1000
        last_commit = bucket_author['last_commit']['value']/1000
        org_name = first['_source']['author_org_name']
        #uuid = first['_source']['author_uuid']
        if datetime.utcfromtimestamp(last_commit).year == year:
            buckets.append({
                    'first_commit': datetime.utcfromtimestamp(first_commit),
                    'last_commit': datetime.utcfromtimestamp(last_commit),
                    'author': author,
                    #'uuid': uuid,
                    'org': org_name
            })
    authors_df = pandas.DataFrame.from_records(buckets)
    authors_df.sort_values(by='first_commit', ascending=False,
                            inplace=True)
    return authors_df

In [4]:
def print_horizontal_bar_chart(df, experience_field, title, min_range = 0):
    
    plotly.offline.init_notebook_mode(connected=True)
    
    experience = list(range(min_range, int(df[experience_field].max()) + 1))
    
    people_count = []
    for exp in experience:
        people_count.append(len(df.loc[df[experience_field] == exp]))
        
    data = [go.Bar(
            x=people_count,
            y=experience,
            orientation = 'h'
    )]
    
    layout = go.Layout(
        barmode='group',
        title= title
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='horizontal-bar')
    
    
def print_horizontal_bar_chart_relative(df, experience_field, title):
    
    plotly.offline.init_notebook_mode(connected=True)
    
    experience = list(range(0, int(df[experience_field].max()) + 1))
    
    people_count = []
    first_count = len(df.loc[df[experience_field] == 0])
    for exp in experience:
        current_count = len(df.loc[df[experience_field] == exp])
        people_count.append(current_count * 100/ first_count)
        
    data = [go.Bar(
            x=people_count,
            y=experience,
            orientation = 'h'
    )]
    
    layout = go.Layout(
        barmode='group',
        title= title
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='horizontal-bar')
    
def print_horizontal_bar_chart_percent(df, experience_field, title):
    
    plotly.offline.init_notebook_mode(connected=True)
    
    experience = list(range(0, int(df[experience_field].max()) + 1))
    
    people_count = []
    cusum = len(df)
    for exp in experience:
        current_count = len(df.loc[df[experience_field] == exp])
        people_count.append(current_count * 100/ cusum)
        
    data = [go.Bar(
            x=people_count,
            y=experience,
            orientation = 'h'
    )]
    
    layout = go.Layout(
        barmode='group',
        title= title
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='horizontal-bar')
    


# Metrics

## Groups of Contributors based on their experience in the Community

Looking at when a given contributor sent her first commit we calculate how long he has been contributing to the community at a given time. We define following groups:
* People with more than **1 month** of experience and less than **6 months**
* People with more than **6 months** of experience and less than **12 months**
* People with more than **1 year** of experience and less than **2 years**
* People with more than **2 years** of experience and less than **4 years**
* People with more than **4 years**


In [5]:
# Define ranges
ranges = [{
           'from': 'now-1M/M',
           'key': '1- Months'
         }, {
           'from': 'now-6M/M',
           'to': 'now-1M/M',
           'key': '1-6 Months'
         }, {
           'from': 'now-12M/M',
           'to': 'now-6M/M',
           'key': '6-12 Months'
         }, {
           'from': 'now-24M/M',
           'to': 'now-12M/M',
           'key':  '1-2 Years'
         }, {
           'from': 'now-48M/M',
           'to': 'now-24M/M',
           'key': '2-4 Years'
         }, {
           'to': 'now-24M/M',
           'key': '4+ Years'
         }]

In [6]:
s = Search(using=es_conn, index='git')
s.params(timeout=30)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs\
    .bucket('experience', 'date_range', field='grimoire_creation_date', ranges=ranges)\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

    #.bucket('org', 'terms', field='author_org_name', size=10)\
    #.bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
#print(s.to_dict())
result = s.execute()

# In case you need to check response, uncomment line below
print(result.to_dict()['aggregations'])


{'experience': {'buckets': [{'contributors': {'value': 0}, 'to_as_string': '2015-05-01T00:00:00.000Z', 'doc_count': 0, 'to': 1430438400000.0, 'key': '4+ Years'}, {'doc_count': 0, 'key': '2-4 Years', 'contributors': {'value': 0}, 'from_as_string': '2013-05-01T00:00:00.000Z', 'from': 1367366400000.0, 'to': 1430438400000.0, 'to_as_string': '2015-05-01T00:00:00.000Z'}, {'doc_count': 73547, 'key': '1-2 Years', 'contributors': {'value': 1265}, 'from_as_string': '2015-05-01T00:00:00.000Z', 'from': 1430438400000.0, 'to': 1462060800000.0, 'to_as_string': '2016-05-01T00:00:00.000Z'}, {'doc_count': 24302, 'key': '6-12 Months', 'contributors': {'value': 789}, 'from_as_string': '2016-05-01T00:00:00.000Z', 'from': 1462060800000.0, 'to': 1477958400000.0, 'to_as_string': '2016-11-01T00:00:00.000Z'}, {'doc_count': 0, 'key': '1-6 Months', 'contributors': {'value': 0}, 'from_as_string': '2016-11-01T00:00:00.000Z', 'from': 1477958400000.0, 'to': 1491004800000.0, 'to_as_string': '2017-04-01T00:00:00.000Z'}

## Time from first to last contrib for authors who made a commit before a given year  

In [7]:
results = []
for i in range(0,4):

    # Buckets by author name, finding first commit for each of them
    s = Search(using=es_conn, index='git')
    s.params(timeout=30)

    # Retrieve commits before given year
    s = s.filter('range', grimoire_creation_date={'lt': 'now-' + str(i) + 'y/y'})

    # Bucketize by uuid and get first and last commit
    s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
        .metric('first', 'top_hits', _source=['author_date', 'author_org_name', 'author_uuid'],
                size=1, sort=[{"author_date": {"order": "asc"}}]) \
        .metric('last_commit', 'max', field='author_date')
    s = s.sort("author_date")
    #       pprint(s.to_dict())
    results.append(s.execute())

In [8]:
authors_dfs = []
for result in results:
    authors_df = get_authors_df(result, author_bucket_field='authors')
    authors_df['active_years'] = (authors_df.last_commit-authors_df.first_commit).astype('timedelta64[Y]')
    authors_dfs.append(authors_df)

authors_dfs


[                                        author        first_commit  \
 3434  1e2828dd94a896d402c26b347cd26489a52a9afe 2016-08-29 12:36:35   
 3118  837e2ec2315a7446cccb438e7768be998e530182 2016-08-26 19:09:12   
 3076  5acf8b9215f9210c90bfba8f829f60466dfd87a1 2016-08-26 18:05:56   
 3423  1c907008c5769f535e5c83b0a249aff4f23d3cd3 2016-08-24 12:39:41   
 2538  4c78e0a3162133de08b507175f4756bd0bcc1a0d 2016-08-23 22:36:46   
 2951  25229e79153818c38ce482cd93f7e0eb6b4567de 2016-08-23 11:20:26   
 3356  128a44bf7ca807ae229f8412f71109d9a2e97385 2016-08-23 10:46:40   
 2952  2527617b37ff60fa5fdfea65b623e86b220ce63e 2016-08-22 12:11:55   
 3067  56ab6606091bb46925658fc5162346171c4eba14 2016-08-17 18:37:30   
 3770  7dff8571555cfed71a1c26a802bd615a73d4fdf0 2016-08-17 17:56:12   
 3903  e4c3eef000736e797237226c78a8c23c043aa4b8 2016-08-16 15:36:58   
 3081  5ebc32fa119e3f2475a5b2cf7487b9138fa08102 2016-08-16 09:31:30   
 2969  29e5f8897fedc189b5127385c46e25733c191b44 2016-08-15 13:28:33   
 2887 

In [9]:
# Plot bar charts for each dataframe
i = 0
for authors_df in authors_dfs:
#    print(author_df['experience_years'].max(), type(author_df['experience_years'].max()))
    print_horizontal_bar_chart(authors_df, 'active_years', title=str(2017 - i))
    i += 1


In [10]:
# Plot bar charts for each dataframe
i = 0
for authors_df in authors_dfs:
#    print(author_df['experience_years'].max(), type(author_df['experience_years'].max()))
    print_horizontal_bar_chart_relative(authors_df, 'active_years', title=str(2017 - i))
    i += 1

In [11]:
# Plot bar charts for each dataframe
i = 0
for authors_df in authors_dfs:
#    print(author_df['experience_years'].max(), type(author_df['experience_years'].max()))
    print_horizontal_bar_chart_percent(authors_df, 'active_years', title=str(2017 - i))
    i += 1

## Time from first to last commit for authors active in a given year

We define an author as **active** iff she made at least one commit within a given year. E.g. an author would be considered active in 2017 if she made a commit after Jan. 1st, 2017 and before Dec. 31st 2017. 

In [12]:
results = []
for i in range(0,4):

    # Buckets by author name, finding first commit for each of them
    s = Search(using=es_conn, index='git')
    s.params(timeout=30)

    # Retrieve commits before given year
    s = s.filter('range', grimoire_creation_date={'lte': 'now-' + str(i) + 'y/y'})

    # Bucketize by uuid and get first and last commit
    s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
        .metric('first', 'top_hits', _source=['author_date', 'author_org_name', 'author_uuid'],
                size=1, sort=[{"author_date": {"order": "asc"}}]) \
        .metric('last_commit', 'max', field='author_date')
    s = s.sort("author_date")
    #       pprint(s.to_dict())
    results.append(s.execute())

In [13]:
authors_dfs = []
year = 2016
for result in results:
    authors_df = get_active_authors_df(result, author_bucket_field='authors', year=year)
    authors_df['active_years'] = (authors_df.last_commit-authors_df.first_commit).astype('timedelta64[Y]')
    authors_dfs.append(authors_df)
    year -= 1

authors_dfs


[                                        author        first_commit  \
 990   1e2828dd94a896d402c26b347cd26489a52a9afe 2016-08-29 12:36:35   
 922   837e2ec2315a7446cccb438e7768be998e530182 2016-08-26 19:09:12   
 911   5acf8b9215f9210c90bfba8f829f60466dfd87a1 2016-08-26 18:05:56   
 987   1c907008c5769f535e5c83b0a249aff4f23d3cd3 2016-08-24 12:39:41   
 802   4c78e0a3162133de08b507175f4756bd0bcc1a0d 2016-08-23 22:36:46   
 884   25229e79153818c38ce482cd93f7e0eb6b4567de 2016-08-23 11:20:26   
 979   128a44bf7ca807ae229f8412f71109d9a2e97385 2016-08-23 10:46:40   
 885   2527617b37ff60fa5fdfea65b623e86b220ce63e 2016-08-22 12:11:55   
 910   56ab6606091bb46925658fc5162346171c4eba14 2016-08-17 18:37:30   
 1056  7dff8571555cfed71a1c26a802bd615a73d4fdf0 2016-08-17 17:56:12   
 1092  e4c3eef000736e797237226c78a8c23c043aa4b8 2016-08-16 15:36:58   
 913   5ebc32fa119e3f2475a5b2cf7487b9138fa08102 2016-08-16 09:31:30   
 888   29e5f8897fedc189b5127385c46e25733c191b44 2016-08-15 13:28:33   
 874  

In [14]:
# Plot bar charts for each dataframe
i = 0
for authors_df in authors_dfs:
#    print(author_df['experience_years'].max(), type(author_df['experience_years'].max()))
    print_horizontal_bar_chart(authors_df, 'active_years', title=str(2016 - i))
    i += 1


## Years of Experience
We consider **12 commits** per year, i.e. one commit per month aprox., as a minimum to add one year of experience to a given author. From this assumption, we build groups of authors by years of experience. As a result, we present a plot with number of people in each group.

To give a more complete idea of how community evolves, we plot snapshots corresponding to different years. Each of them will take all commits sent until the given year, and calculate years of experience for all authors in that slice.

In [15]:
###
## GET COMMITS BY YEAR AND AUTHOR
###

results = []
min_commits = 1

for i in range(0,10):

    # Buckets by author name, finding first commit for each of them
    s = create_search(source='git')
    
    # Retrieve commits before given year
    s = s.filter('range', grimoire_creation_date={'lte': 'now-' + str(i) + 'y/y'})

    # Bucketize by uuid and get first and last commit
    s.aggs.bucket('time', 'date_histogram', field='grimoire_creation_date', interval='year')\
        .bucket('authors', 'terms', field='author_uuid', size=100000, min_doc_count=min_commits) \
        .metric('commits', 'cardinality', field='hash', precision_threshold=1000)

    r = s.execute()
    # In case you need to check response, uncomment line below
    #print(len(r.to_dict()['aggregations']['time']['buckets']))
        
    results.append(r)
    
#results

In [27]:
###
## CREATE A DF CONTAINING, FOR EACH AUTHOR UUID, COUNT OF YEARS OF EXPERIENCE (YEARS
## WITH MORE THAN 12 COMMITS MADE) AND LAST YEAR ACTIVE
###
exp_df_list = []
year = 2017

for result in results:
    exp_df = ut.to_df_by_time(result, 'Author', 'Time', 'Commits', 'authors', 'time', 'commits')
    exp_df['Time'] = exp_df['Time'].apply(lambda x: str(pandas.Period(x,'A')))
    
    ## ACTIVE CONDITION
    ## Filter those having less than 12 commits per year
    exp_df = exp_df[exp_df['Commits'] >= 12]
    
    ## Group by author, get MAX YEAR and NUMBER OF ROWS FOR THE GIVEN AUTHOR
    exp_df = exp_df.groupby(['Author']).agg({'Time': 'max', 'Commits': 'count'})
    ## Filter those whose last active year is not the one we want
    exp_df = exp_df[exp_df['Time'] == str(year)]
    
    exp_df['exp'] = exp_df['Commits']
    exp_df['last_active'] = exp_df['Time']
    exp_df= exp_df.drop('Commits', axis=1)
    exp_df = exp_df.drop('Time', axis=1)
    
    exp_df_list.append(exp_df)
    
    year -= 1

exp_df_list


[Empty DataFrame
 Columns: [exp, last_active]
 Index: [],                                           exp last_active
 Author                                                   
 00099b1df1503d0b13a2bb22f6e5ca412b625790    2        2016
 00156a67e01c5c6536f01b4f1093eef6a23fc910    4        2016
 0030321a29d9015f998c45fca4191ab05d256df6    4        2016
 0044a6fac7c86283b03d0e9f18d4db5ece6caa56    4        2016
 005587a4dfceedd91b507f088961ae5a6491c673    5        2016
 0068c3fe62aa7b13a3292eba16c802134403b339   10        2016
 007b1527cdd70f11735145d3f71d4d8271dc920e    2        2016
 0081a0dd40fab20c07c30002fb4593fdcebe659a    3        2016
 00821778f062c73c39bb3822a669761ec42779de    4        2016
 008dfbea15682d26e85bf3a4e86a791447499137    7        2016
 008e53343d1eca2fc73d3ec677ce1036806dcda1   14        2016
 0099ba92796524f0602dddad322eae201fb55193   10        2016
 00a14280350ab198faeae04ff05c8ab6b40923dc    6        2016
 00a97cf25feaef5ca0c50466dd487c3d57c7e791    7        2016

In [28]:
# Plot bar charts for each dataframe
i = 0
for exp_df in exp_df_list:
    if not exp_df.empty:
        print_horizontal_bar_chart(exp_df, 'exp', title=str(2017 - i), min_range=1)
    i += 1

In [29]:
exp_groups_evo_df = pandas.DataFrame(columns=['last_active', 'exp', 'count'])

for exp_df in exp_df_list:
    
    if exp_df.empty:
        continue
    
    experience = list(range(1, int(exp_df['exp'].max()) + 1))
    
    last_active = exp_df['last_active'].unique()[0]
    for exp in experience:
        count = len(exp_df.loc[exp_df['exp'] == exp])
        #print(last_active, exp, count)
        exp_groups_evo_df.loc[len(exp_groups_evo_df)] = [last_active, exp, count]
        
print('Max. Exp: ', exp_groups_evo_df['exp'].max(), 'Max. Count: ',  exp_groups_evo_df['count'].max())
exp_groups_evo_df
    
    
    

Max. Exp:  16.0 Max. Count:  195.0


,last_active,exp,count
0,2016,1.0,144.0
1,2016,2.0,84.0
2,2016,3.0,72.0
3,2016,4.0,45.0
4,2016,5.0,41.0
5,2016,6.0,42.0
6,2016,7.0,21.0
7,2016,8.0,9.0
8,2016,9.0,8.0
9,2016,10.0,3.0


In [30]:
exp_groups_evo_df = pandas.DataFrame(columns=['exp'])

for exp_df in exp_df_list:
    
    if exp_df.empty:
        continue
    
    year = exp_df['last_active'].unique()[0]
    exp_groups_df = pandas.DataFrame(columns=['exp', year])
    
    experience = list(range(1, int(exp_df['exp'].max()) + 1))
    
    for exp in experience:
        count = len(exp_df.loc[exp_df['exp'] == exp])
        exp_groups_df.loc[len(exp_groups_df)] = [exp, count]

    exp_groups_evo_df = exp_groups_evo_df.merge(exp_groups_df, on='exp', how='outer')


# Fill Nan with 0's
exp_groups_evo_df = exp_groups_evo_df.fillna(0)

# Reorder columns
exp_groups_evo_df = exp_groups_evo_df.set_index('exp')
exp_groups_evo_df = exp_groups_evo_df.sort_index(axis=1)


#print('Max. Exp: ', exp_groups_evo_df['exp'].max(), 'Max. Count: ')
exp_groups_evo_df

,2008,2009,2010,2011,2012,2013,2014,2015,2016
exp,,,,,,,,,
1.0,138.0,105.0,139.0,195.0,168.0,137.0,183.0,157.0,144.0
2.0,79.0,109.0,79.0,104.0,145.0,105.0,82.0,116.0,84.0
3.0,70.0,62.0,86.0,55.0,64.0,89.0,80.0,61.0,72.0
4.0,41.0,49.0,45.0,55.0,40.0,43.0,66.0,63.0,45.0
5.0,19.0,37.0,40.0,33.0,30.0,28.0,34.0,55.0,41.0
6.0,8.0,17.0,27.0,33.0,27.0,23.0,18.0,24.0,42.0
7.0,8.0,7.0,13.0,22.0,20.0,15.0,14.0,9.0,21.0
8.0,15.0,5.0,8.0,9.0,17.0,11.0,9.0,10.0,9.0
9.0,0.0,11.0,3.0,7.0,8.0,12.0,9.0,6.0,8.0


In [31]:
plotly.offline.init_notebook_mode(connected=True)

data = []
for exp in exp_groups_evo_df.index.values:
    #print(exp, '\n', exp_groups_evo_df.loc[exp].tolist(), '\n', exp_groups_evo_df.loc[exp].index.values)
    data.append(
        go.Scatter(
            x = exp_groups_evo_df.loc[exp].index.values,
            y = exp_groups_evo_df.loc[exp].tolist(),
            mode = 'lines+markers',
            name = str(int(exp)) + ' years'
        )
    )
    


plotly.offline.iplot(data, filename='line-mode')    

## Retention by experience
Next table shows percentage of people remaining in the community for each experience group. It is calculated not by following individuals, but comparing total number of people in each group.

To read the table have into account that each number corresponds to percentage of people remaining in a given group (e.g. 2 years of experience in 2010) with respect to same group during previous year (i.e. 1 year of experience in 2009).

So, if we look at cell (3.0, 2010) we can read it as number of people with **3** years of experience in **2010** represents a 78.90% of people having **2** years of experience in **2009**. Note that both groups are in fact the same as they evolve through time, increasing their years of experience. 

In [34]:
exp_groups_evo_diff_df = pandas.DataFrame()

for exp in exp_groups_evo_df.index.values:
    #print(exp, '\n', exp_groups_evo_df.loc[exp].tolist(), '\n', exp_groups_evo_df.loc[exp].index.values)
    
    cols = list(exp_groups_evo_df)
    min_col = int(cols[0])
    
    #print(exp - 1)
    for col in list(exp_groups_evo_df): 
        current_val = exp_groups_evo_df.get_value(exp, col)
        prev_row = exp - 1
        prev_col= int(col) - 1
        if prev_row > 0 and prev_col >= min_col:
            prev_val = exp_groups_evo_df.get_value(prev_row, str(prev_col))
            #print(col, prev_val, current_val, prev_val - current_val)
            if prev_val == 0:
                percent = 0
            else:
                percent = current_val * 100 / prev_val
            exp_groups_evo_diff_df.set_value(exp, col, round(percent, 2))

exp_groups_evo_diff_df

    


,2009,2010,2011,2012,2013,2014,2015,2016
2.0,78.99,75.24,74.82,74.36,62.50,59.85,63.39,53.50
3.0,78.48,78.90,69.62,61.54,61.38,76.19,74.39,62.07
4.0,70.00,72.58,63.95,72.73,67.19,74.16,78.75,73.77
5.0,90.24,81.63,73.33,54.55,70.00,79.07,83.33,65.08
6.0,89.47,72.97,82.50,81.82,76.67,64.29,70.59,76.36
7.0,87.50,76.47,81.48,60.61,55.56,60.87,50.00,87.50
8.0,62.50,114.29,69.23,77.27,55.00,60.00,71.43,100.00
9.0,73.33,60.00,87.50,88.89,70.59,81.82,66.67,80.00
10.0,0.00,72.73,100.00,71.43,37.50,58.33,77.78,50.00
11.0,0.00,0.00,87.50,66.67,80.00,33.33,71.43,57.14


### Following evolution of groups

Another way of visualizing this is selecting a group from a particular year and following its evolution throug time. This way, we will get number of people belonging to that group through time.

Next table shows evolution of groups from a given year, in this case **2008**. So first row shows how the 138 people who had 1 year of experience in 2008 have evolved throug years, losing people year by year until 2016, when there are only 8 of them. 

In [46]:
group_evo_df = pandas.DataFrame()

years = list(exp_groups_evo_df)

# Group we want to follow
first_year = 2008


for exp_group in exp_groups_evo_df.index.values:
    
    exp_index = exp_group

    for year in years:
        if int(year) < first_year:
            continue

        if exp_index > exp_groups_evo_df.index.values.max():
            break

        people_count = exp_groups_evo_df.get_value(exp_index, year)
        #print(exp_index, year, people_count)
        group_evo_df.set_value(exp_group, year, people_count)
        exp_index += 1

group_evo_df = group_evo_df.fillna(0)
group_evo_df


,2008,2009,2010,2011,2012,2013,2014,2015,2016
1.0,138.0,109.0,86.0,55.0,30.0,23.0,14.0,10.0,8.0
2.0,79.0,62.0,45.0,33.0,27.0,15.0,9.0,6.0,3.0
3.0,70.0,49.0,40.0,33.0,20.0,11.0,9.0,7.0,4.0
4.0,41.0,37.0,27.0,22.0,17.0,12.0,7.0,5.0,3.0
5.0,19.0,17.0,13.0,9.0,8.0,3.0,1.0,2.0,2.0
6.0,8.0,7.0,8.0,7.0,5.0,4.0,4.0,4.0,4.0
7.0,8.0,5.0,3.0,3.0,2.0,0.0,0.0,1.0,1.0
8.0,15.0,11.0,8.0,7.0,5.0,5.0,4.0,4.0,3.0
9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can look at the same data in percentages over previous years, i.e. how many people are retained in a group from one year to the following:

In [53]:
group_evo_percent_df = pandas.DataFrame()

years = list(exp_groups_evo_df)

# Group we want to follow
first_year = 2008


for exp_group in exp_groups_evo_df.index.values:
    
    exp_index = exp_group

    for year in years:
        if int(year) < first_year:
            continue

        if exp_index > exp_groups_evo_df.index.values.max():
            break
        
        current_val = exp_groups_evo_df.get_value(exp_index, year)
        
        if int(year) == first_year:
            prev_val = current_val
        
        if prev_val == 0:
            percent = current_val * 100
        else:
            percent = current_val * 100 / prev_val
        group_evo_percent_df.set_value(exp_group, year, round(percent, 2))
        
        exp_index += 1
        prev_val = current_val
              
        

group_evo_percent_df = group_evo_percent_df.fillna(0)
group_evo_percent_df

,2008,2009,2010,2011,2012,2013,2014,2015,2016
1.0,100.0,78.99,78.90,63.95,54.55,76.67,60.87,71.43,80.00
2.0,100.0,78.48,72.58,73.33,81.82,55.56,60.00,66.67,50.00
3.0,100.0,70.00,81.63,82.50,60.61,55.00,81.82,77.78,57.14
4.0,100.0,90.24,72.97,81.48,77.27,70.59,58.33,71.43,60.00
5.0,100.0,89.47,76.47,69.23,88.89,37.50,33.33,200.00,100.00
6.0,100.0,87.50,114.29,87.50,71.43,80.00,100.00,100.00,100.00
7.0,100.0,62.50,60.00,100.00,66.67,0.00,0.00,100.00,100.00
8.0,100.0,73.33,72.73,87.50,71.43,100.00,80.00,100.00,75.00
9.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
